#  <center> SolvaNET: Final Project <center>

<center> 10.C51 <center>

In [1]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 32.7 MB/s eta 0:00:00


In [2]:
!wget https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps6-sfe/data/molecule_props.csv
!wget https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps6-sfe/data/solvation_train.csv
!wget https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps6-sfe/data/solvation_test.csv

--2024-05-10 01:51:25--  https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps6-sfe/data/molecule_props.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26739 (26K) [text/plain]
Saving to: ‘molecule_props.csv’

molecule_props.csv  100%[===================>]  26.11K  --.-KB/s    in 0s      

2024-05-10 01:51:25 (141 MB/s) - ‘molecule_props.csv’ saved [26739/26739]

--2024-05-10 01:51:25--  https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps6-sfe/data/solvation_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [3]:
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors,Crippen
import numpy as np

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



from sklearn.ensemble import RandomForestRegressor

from torch.utils.data import Dataset, DataLoader

!pip install torch_geometric
import torch_geometric
from tqdm import tqdm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00


## Part 1: Baseline Regression Methods

### Part 1.1: Prepare Dataset

In [4]:
# load data
train_data = pd.read_csv('./solvation_train.csv')
mol_prop = pd.read_csv('./molecule_props.csv')
test_data = pd.read_csv('./solvation_test.csv')
print(train_data)
print(mol_prop)
print(test_data)

               Solvent             Solute   logK  deltaG_solv
0      C(Cl)(Cl)(Cl)Cl       c1ccc(cc1)CO  4.830     -0.28574
1         c1ccc(cc1)Br               CCCO  2.760     -0.16328
2         CC(C)CC(=O)C       CC(Cl)(Cl)Cl  3.134     -0.18541
3           CC(C)(C)OC  C(C(F)(F)F)(Cl)Br  3.440     -0.20351
4              CC(=O)C         CC(C)(C)Cl  2.540     -0.15026
...                ...                ...    ...          ...
3653          CN(C)C=O              C(F)F  1.240     -0.07336
3654         Cc1ccccc1       CCCCCC(=O)OC  4.676     -0.27663
3655  CCCCCCCCCCCCCCCC            CCCCCCN  3.557     -0.21043
3656   C(Cl)(Cl)(Cl)Cl      c1ccc(cc1)C=O  4.480     -0.26503
3657  CCCCCCCCCCCCCCCC       CC(C)CC(=O)O  3.300     -0.19523

[3658 rows x 4 columns]
             SMILES  chiral  mol_weight  polarizability    dipole
0           CCC(C)O       1       74.12            8.81  1.646671
1          C1COCCO1       0       88.11            8.70  1.477152
2         CCCCCCCCO       0      

Some utility functions for you to generate features.

Generate fingerprints (e.g. a Morgan fingerprint).

In [5]:
# Get molecular feature from RDkit
smiles = 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'

# define Mol object
mol = Chem.MolFromSmiles(smiles)

# get morgan fingerprint
# obtain a 512 bit fingperint, with radius 2
fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512)

# convert to numpy array
fp_array = np.zeros((1,), int)
DataStructs.ConvertToNumpyArray(fp, fp_array)

Generate various chemical properties.

In [6]:
MolWt = Descriptors.ExactMolWt(mol)
TPSA = Chem.rdMolDescriptors.CalcTPSA(mol) #Topological Polar Surface Area
nRotB = Descriptors.NumRotatableBonds(mol) #Number of rotable bonds
HBD = Descriptors.NumHDonors(mol) #Number of H bond donors
HBA = Descriptors.NumHAcceptors(mol) #Number of H bond acceptors
logP = Descriptors.MolLogP(mol) #LogP

Create a feature set with concatenated physical descriptors.

In [7]:
#creating numpy array of one hot encoded labels for solvent-solute pairs
#similar to pset 2

#TRAIN DATA
# Extract columns solvents and solutes
col_solvent_tr = train_data['Solvent']
col_solute_tr = train_data['Solute']
y_tr = train_data['logK']

# Initialize LabelBinarizer
label_binarizer = LabelBinarizer()
label_binarizer.fit(mol_prop['SMILES'])

# Binarize column A and B
bit_vectors_solvent_tr = label_binarizer.transform(col_solvent_tr)
bit_vectors_solute_tr = label_binarizer.transform(col_solute_tr)

# Stack bit vectors horizontally
stacked_train = np.hstack((bit_vectors_solvent_tr, bit_vectors_solute_tr))

#TEST DATA
# Extract columns solvents and solutes
col_solvent_te = test_data['Solvent']
col_solute_te = test_data['Solute']


bit_vectors_solvent_te = label_binarizer.transform(col_solvent_te)
bit_vectors_solute_te = label_binarizer.transform(col_solute_te)

stacked_test = np.hstack((bit_vectors_solvent_te, bit_vectors_solute_te))

print(stacked_train.shape)
print(stacked_test)



(3658, 1314)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


create numpy arrays of raw descriptors

In [8]:
# helper functions

def get_item(smiles_list, mol_prop_data, feature):
  '''
  smiles_list: list of SMILES strings
  mol_prop_data: pandas DataFrame containing molecular properties
  feature: string specifying the molecular property to extract

  returns: numpy array of molecular properties
  '''
  # Initialize an empty list to store the molecular properties
  prop_list = []

  # Iterate over each SMILES string
  for smiles in smiles_list:
      # Find the row corresponding to the SMILES string in the DataFrame
      row = mol_prop_data[mol_prop_data['SMILES'] == smiles]

      # Check if the SMILES string exists in the DataFrame
      if not row.empty:
          # Extract the value of the specified molecular property
          prop_value = row[feature].values[0]
          # Append the property value to the list
          prop_list.append(prop_value)
      else:
          # If the SMILES string is not found, append NaN (or any other placeholder)
          prop_list.append(np.nan)

  # Convert the list of molecular properties to a numpy array
  prop_array = np.array(prop_list)

  return prop_array

#deal with NaN
def fill_nan_with_mean(array):
  '''
  array: numpy array of raw descriptors
  returns: numpy array with NaN values filled with the mean of the non-NaN values
  '''
  # Calculate the mean of the non-NaN values
  mean_val = np.nanmean(array[~np.isnan(array)])

  # Replace NaN values with the mean
  array[np.isnan(array)] = mean_val

  return array

from rdkit.Chem import Descriptors, rdMolDescriptors
def get_raw_desc(smiles_list, feature):
    '''
    smiles_list: list of SMILES strings
    feature: string specifying the molecular property to extract

    returns: numpy array of molecular properties
    '''
    # Pre-calculate molecule objects for all SMILES strings
    mol_list = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

    # Initialize a dictionary mapping features to descriptor functions
    descriptor_functions = {
        'TPSA': rdMolDescriptors.CalcTPSA,
        'nRotB': Descriptors.NumRotatableBonds,
        'HBD': Descriptors.NumHDonors,
        'HBA': Descriptors.NumHAcceptors,
        'logP': Descriptors.MolLogP
    }

    # Get the appropriate descriptor function for the specified feature
    descriptor_function = descriptor_functions.get(feature)

    if descriptor_function is None:
        raise ValueError("Invalid feature name")

    # Calculate molecular properties using the selected descriptor function
    prop_list = [descriptor_function(mol) for mol in mol_list if mol is not None]

    # Convert the list of molecular properties to a numpy array
    prop_array = np.array(prop_list)

    return prop_array


In [9]:
#TRAIN DATA
# create numpy arrays for solvent
solvent_mw_tr = fill_nan_with_mean(get_item(train_data['Solvent'], mol_prop, 'mol_weight'))
solvent_polarity_tr = fill_nan_with_mean(get_item(train_data['Solvent'], mol_prop, 'polarizability'))
solvent_dipole_tr = fill_nan_with_mean(get_item(train_data['Solvent'], mol_prop, 'dipole'))
solvent_chiral_tr = fill_nan_with_mean(get_item(train_data['Solvent'], mol_prop, 'chiral'))
solvent_tpsa_tr = fill_nan_with_mean(get_raw_desc(train_data['Solvent'], 'TPSA'))
solvent_nrotb_tr = fill_nan_with_mean(get_raw_desc(train_data['Solvent'], 'nRotB'))
solvent_hbd_tr = fill_nan_with_mean(get_raw_desc(train_data['Solvent'], 'HBD'))
solvent_hba_tr = fill_nan_with_mean(get_raw_desc(train_data['Solvent'], 'HBA'))
solvent_logp_tr = fill_nan_with_mean(get_raw_desc(train_data['Solvent'], 'logP'))

#create numpy arrays for solute
solute_mw_tr = fill_nan_with_mean(get_item(train_data['Solute'], mol_prop, 'mol_weight'))
solute_polarity_tr = fill_nan_with_mean(get_item(train_data['Solute'], mol_prop, 'polarizability'))
solute_dipole_tr = fill_nan_with_mean(get_item(train_data['Solute'], mol_prop, 'dipole'))
solute_chiral_tr = fill_nan_with_mean(get_item(train_data['Solute'], mol_prop, 'chiral'))
solute_tpsa_tr = fill_nan_with_mean(get_raw_desc(train_data['Solute'], 'TPSA'))
solute_nrotb_tr = fill_nan_with_mean(get_raw_desc(train_data['Solute'], 'nRotB'))
solute_hbd_tr = fill_nan_with_mean(get_raw_desc(train_data['Solute'], 'HBD'))
solute_hba_tr = fill_nan_with_mean(get_raw_desc(train_data['Solute'], 'HBA'))
solute_logp_tr = fill_nan_with_mean(get_raw_desc(train_data['Solute'], 'logP'))



#TEST DATA
solvent_mw_te = fill_nan_with_mean(get_item(test_data['Solvent'], mol_prop, 'mol_weight'))
solvent_polarity_te = fill_nan_with_mean(get_item(test_data['Solvent'], mol_prop, 'polarizability'))
solvent_dipole_te = fill_nan_with_mean(get_item(test_data['Solvent'], mol_prop, 'dipole'))
solvent_chiral_te = fill_nan_with_mean(get_item(test_data['Solvent'], mol_prop, 'chiral'))
solvent_tpsa_te = fill_nan_with_mean(get_raw_desc(test_data['Solvent'], 'TPSA'))
solvent_nrotb_te = fill_nan_with_mean(get_raw_desc(test_data['Solvent'], 'nRotB'))
solvent_hbd_te = fill_nan_with_mean(get_raw_desc(test_data['Solvent'], 'HBD'))
solvent_hba_te = fill_nan_with_mean(get_raw_desc(test_data['Solvent'], 'HBA'))
solvent_logp_te = fill_nan_with_mean(get_raw_desc(test_data['Solvent'], 'logP'))

solute_mw_te = fill_nan_with_mean(get_item(test_data['Solute'], mol_prop, 'mol_weight'))
solute_polarity_te = fill_nan_with_mean(get_item(test_data['Solute'], mol_prop, 'polarizability'))
solute_dipole_te = fill_nan_with_mean(get_item(test_data['Solute'], mol_prop, 'dipole'))
solute_chiral_te = fill_nan_with_mean(get_item(test_data['Solute'], mol_prop, 'chiral'))
solute_tpsa_te = fill_nan_with_mean(get_raw_desc(test_data['Solute'], 'TPSA'))
solute_nrotb_te = fill_nan_with_mean(get_raw_desc(test_data['Solute'], 'nRotB'))
solute_hbd_te = fill_nan_with_mean(get_raw_desc(test_data['Solute'], 'HBD'))
solute_hba_te = fill_nan_with_mean(get_raw_desc(test_data['Solute'], 'HBA'))
solute_logp_te = fill_nan_with_mean(get_raw_desc(test_data['Solute'], 'logP'))



[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not removing hydrogen atom without neighbors
[01:52:49] WARNING: not r

In [10]:
#combine numpy arrays
#TRAIN DATA
X_train_uns = np.column_stack((solvent_mw_tr, solute_mw_tr,
                               solvent_polarity_tr, solute_polarity_tr,
                               solvent_dipole_tr, solute_dipole_tr,
                               solvent_chiral_tr, solute_chiral_tr,
                               solvent_tpsa_tr, solute_tpsa_tr,
                               solvent_nrotb_tr, solute_nrotb_tr,
                               solvent_hbd_tr, solute_hbd_tr,
                               solvent_hba_tr, solute_hba_tr,
                               solvent_logp_tr, solute_logp_tr))

#TEST DATA
X_test_uns = np.column_stack((solvent_mw_te, solute_mw_te,
                              solvent_polarity_te, solute_polarity_te,
                              solvent_dipole_te, solute_dipole_te,
                              solvent_chiral_te, solute_chiral_te,
                              solvent_tpsa_te, solute_tpsa_te,
                              solvent_nrotb_te, solute_nrotb_te,
                              solvent_hbd_te, solute_hbd_te,
                              solvent_hba_te, solute_hba_te,
                              solvent_logp_te, solute_logp_te))

print(X_train_uns)


[[153.82       108.14        10.61       ...   1.           2.5529
    1.1789    ]
 [157.01        60.1         13.25       ...   1.           2.4491
    0.3887    ]
 [100.16       133.4         12.06       ...   0.           1.6215
    2.3765    ]
 ...
 [226.44       101.19        11.23448246 ...   1.           6.4876
    1.5254    ]
 [153.82       106.12        10.61       ...   1.           2.5529
    1.4991    ]
 [226.44       102.13        11.23448246 ...   1.           6.4876
    1.1171    ]]


In [11]:
#normalize

#TRAIN DATA
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_1 = scaler.fit_transform(X_train_uns)
X_test_final = scaler.transform(X_test_uns)
# print(X_train_1)
# print(X_test_final)
print(X_test_final.shape)


(1569, 18)


### Morgan fingerprint data

In [24]:
# helper function

def get_fingerprint_features(smiles_list):
  fingerprint_list = []
  for smiles in smiles_list:

    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=100)
    fp_array = np.zeros((1,), int)
    DataStructs.ConvertToNumpyArray(fp, fp_array)
    fingerprint_list.append(fp_array)
  fingerprint_array = np.array(fingerprint_list)
  return fingerprint_array


In [25]:
# great fp data as numpy arrays

# TRAIN DATA
solvent_fp_tr = get_fingerprint_features(train_data['Solvent'])
solute_fp_tr = get_fingerprint_features(train_data['Solute'])

# TEST DATA
solvent_fp_te = get_fingerprint_features(test_data['Solvent'])
solute_fp_te = get_fingerprint_features(test_data['Solute'])

[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not removing hydrogen atom without neighbors
[02:20:26] WARNING: not r

In [26]:
X_train_fp = np.column_stack([X_train_1, solvent_fp_tr, solute_fp_tr])

X_test_fp = np.column_stack([X_test_final, solvent_fp_te, solute_fp_te])
print(X_train_fp.shape)


(3658, 218)


### Mordred data

In [12]:
pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.3 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176720 sha256=fe9c7f1497eddc843945497bff0c15db1bda7476243f50ba1a5c8bcb5e781840
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-c

In [13]:
from mordred import Calculator, descriptors
from rdkit import Chem

# Load your molecule (assuming it's in SMILES format)
mol = Chem.MolFromSmiles("CCO")

# Create a descriptor calculator instance
calc = Calculator(descriptors, ignore_3D=True)  # Ignore 3D descriptors for now

# Calculate descriptors
descriptors = calc(mol)

# Extract specific descriptors
hydrophilic_surface_area = descriptors['SlogP_VSA1']
hydrophobic_surface_area = descriptors['SMR_VSA1']
topological_descriptors_1 = descriptors['BalabanJ']
topological_descriptors_2 = descriptors['BertzCT']
# Extract more descriptors as needed

# Print or use the calculated descriptors
print("Hydrophilic Surface Area:", hydrophilic_surface_area)
print("Hydrophobic Surface Area:", hydrophobic_surface_area)
print("Topological Descriptors:", topological_descriptors_1)
print("Topological Descriptors:", topological_descriptors_2)

from rdkit.Chem import Descriptors3D
# Generate 3D conformer
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# Calculate 3D shape descriptors
volume = Descriptors3D.Asphericity(mol)
spherocity = Descriptors3D.SpherocityIndex(mol)


# Print or use the calculated descriptors
print("Volume:", volume)
print("Spherocity:", spherocity)



Hydrophilic Surface Area: 0.0
Hydrophobic Surface Area: 5.106527394840706
Topological Descriptors: 1.6329931618554523
Topological Descriptors: 2.7548875021634682
Volume: 0.4297125471872351
Spherocity: 0.4966664137659443


In [21]:
# helper function for mordred data
import numpy as np
from rdkit import Chem
from mordred import Calculator, descriptors

def get_mordred_desc(smiles_list, feature):
    '''
    smiles_list: list of SMILES strings
    feature: string specifying the molecular property to extract

    returns: numpy array of molecular properties
    '''
    mordred_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        # Create a descriptor calculator instance
        calc = Calculator(descriptors, ignore_3D=True)  # Ignore 3D descriptors for now

        # Calculate descriptors
        descriptors_1 = calc(mol)
        if feature == 'SlogP_VSA1':
          mordred_list.append(descriptors_1['SlogP_VSA1'])
        if feature == 'SMR_VSA1':
          mordred_list.append(descriptors_1['SMR_VSA1'])
        if feature == 'BalabanJ':
          mordred_list.append(descriptors_1['BalabanJ'])
        if feature == 'BertzCT':
          mordred_list.append(descriptors_1['BertzCT'])
    return np.array(mordred_list)

def mordred_geometric(smiles_list, feature):
    '''
    smiles_list: list of SMILES strings
    feature: string specifying the 3D geometric property to calculate

    returns: numpy array of 3D geometric properties
    '''
    mordred_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue  # Skip invalid molecules

        # Generate 3D conformer
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        AllChem.UFFOptimizeMolecule(mol)

        if feature == 'Asphericity':
            mordred_list.append(Descriptors3D.Asphericity(mol))
        elif feature == 'SpherocityIndex':
            mordred_list.append(Descriptors3D.SpherocityIndex(mol))

    return np.array(mordred_list)

In [22]:
solvent_mord1_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solvent'], 'SlogP_VSA1'))
solvent_mord2_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solvent'], 'SMR_VSA1'))
solvent_mord3_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solvent'], 'BalabanJ'))
solvent_mord4_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solvent'], 'BertzCT'))
solvent_3d_tr = fill_nan_with_mean(mordred_geometric(train_data['Solvent'], 'Asphericity'))
solvent_4d_tr = fill_nan_with_mean(mordred_geometric(train_data['Solvent'], 'SpherocityIndex'))

solute_mord1_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solute'], 'SlogP_VSA1'))
solute_mord2_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solute'], 'SMR_VSA1'))
solute_mord3_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solute'], 'BalabanJ'))
solute_mord4_tr = fill_nan_with_mean(get_mordred_desc(train_data['Solute'], 'BertzCT'))
solute_3d_tr = fill_nan_with_mean(mordred_geometric(train_data['Solute'], 'Asphericity'))
solute_4d_tr = fill_nan_with_mean(mordred_geometric(train_data['Solute'], 'SpherocityIndex'))

solvent_mord1_te = fill_nan_with_mean(get_mordred_desc(test_data['Solvent'], 'SlogP_VSA1'))
solvent_mord2_te = fill_nan_with_mean(get_mordred_desc(test_data['Solvent'], 'SMR_VSA1'))
solvent_mord3_te = fill_nan_with_mean(get_mordred_desc(test_data['Solvent'], 'BalabanJ'))
solvent_mord4_te = fill_nan_with_mean(get_mordred_desc(test_data['Solvent'], 'BertzCT'))
solvent_3d_te = fill_nan_with_mean(mordred_geometric(test_data['Solvent'], 'Asphericity'))
solvent_4d_te = fill_nan_with_mean(mordred_geometric(test_data['Solvent'], 'SpherocityIndex'))

solute_mord1_te = fill_nan_with_mean(get_mordred_desc(test_data['Solute'], 'SlogP_VSA1'))
solute_mord2_te = fill_nan_with_mean(get_mordred_desc(test_data['Solute'], 'SMR_VSA1'))
solute_mord3_te = fill_nan_with_mean(get_mordred_desc(test_data['Solute'], 'BalabanJ'))
solute_mord4_te = fill_nan_with_mean(get_mordred_desc(test_data['Solute'], 'BertzCT'))
solute_3d_te = fill_nan_with_mean(mordred_geometric(test_data['Solute'], 'Asphericity'))
solute_4d_te = fill_nan_with_mean(mordred_geometric(test_data['Solute'], 'SpherocityIndex'))

KeyboardInterrupt: 

### Choice of features explanation

In [93]:
# First I started by extracting information from mol_prop data provided.
# Then, I also added more information from rdkit.
# all of the features used has some sort of effect on solvation free energy,
# so, it is important to get as much as feature as we can.
# next, I also added morgan fingerprint data with nbits size 200
# as we discussed in one of the psets, morgan fingerprint data can provide crucial informations for chemical compounds
# each chemical feature of solvent and solute are represented as columns

# I also tried adding mordred data
# However, the data collection for mordred was too slow, so I skipped that part
# since it would be inefficient.

### 1.2 Linear Regression

Train a linear regression model and report a 5-fold cross-validated R^2.

In [49]:



# Create a linear regression model
model_linear = LinearRegression()

# Train the model
model_linear.fit(X_train_1, y_tr)

# Perform 5-fold cross-validation for R2 score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2_scores = cross_val_score(model_linear, X_train_1, y_tr, cv=kf, scoring='r2')



# Report the 5-fold cross-validated R2 score
print("Cross-Validated R2 Scores:", cv_r2_scores)
print("Mean CV R2 Score:", np.mean(cv_r2_scores))



Cross-Validated R2 Scores: [0.79986372 0.82761602 0.79202493 0.8228516  0.83742855]
Mean CV R2 Score: 0.8159569660696908


### 1.3  MLP Regression

Train an MLP regression model and report a 5-fold cross-validated R^2.

In [44]:


# Assuming you have a NumPy array 'data_array' of shape (1569, 18) for input features
# and a NumPy array 'target_array' of shape (1569, 1) for target values
input_tensor = torch.tensor(X_train_fp, dtype=torch.float32)
target_tensor = torch.tensor(y_tr, dtype=torch.float32)

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, input_tensor, target_tensor):
        self.input_data = input_tensor
        self.target_data = target_tensor

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return self.input_data[idx], self.target_data[idx]

# Create DataLoader
batch_size = 32
custom_dataset = CustomDataset(input_tensor, target_tensor)
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Define MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(218, 128)  # Adjust input size to 18 for X_tr_1
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate model
mlp_model = MLP()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)

# Training loop
num_epochs = 100 # Set an appropriate number of epochs
for epoch in range(num_epochs):
    for batch_input, batch_target in data_loader:
        optimizer.zero_grad()
        outputs = mlp_model(batch_input)
        loss = criterion(outputs, batch_target)
        loss.backward()
        optimizer.step()

from sklearn.neural_network import MLPRegressor

# Instantiate the regressor

mlp_model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000)

# Train the model
# Perform 5-fold cross-validation for R^2 score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores = cross_val_score(mlp_model, input_tensor, target_tensor, scoring='r2', cv=kf)

# Print the results
print("5-fold Cross-Validated R^2 Scores:", r2_scores)
print("Mean R^2 Score:", np.mean(r2_scores))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


5-fold Cross-Validated R^2 Scores: [0.95655047 0.95197437 0.96908507 0.95749992 0.95437971]
Mean R^2 Score: 0.9578979047050243


Explanation

In [45]:
# running code for choosing hyperparameters took too long for mlp model
# First I tried X_tr_1 (which is the feature set with chemical properties from rdkit) and got mean cv R^2 score of 0.935
# then I updated my data by adding morgan fingerprint data, and this time I got mean R^2 score 0.956
# I also tried adding more layers by changing it from 64 to 128 and got R^2 score of 0.953
# tried changing num_epochs from 500 to 100 at 128 layer size and got 0.9579
# tried num_epoch = 250 and layer size 128, got 0.9545

# In conclusion, the best result that I obtained was using fingerprint data, num_epochs=100, and layer size (218, 128) and (128, 1)
# I submitted my prediction to Kaggle and it passed the mlp baseline



## Part 2: Machine Learning Competition and Report

You can start a new notebook here to put all your models.

In [50]:
def save_submission(prediction, filename):
    '''
    Utility function to dump a submission file.

    prediction (numpy.array): 1d numpy array contains your prediction
    filename (str): file path to where you want to save the result
    '''
    sub = pd.DataFrame( {'index': list(range(len(prediction))), 'logK': prediction } )
    sub.to_csv(filename, index=False)



## 2.1 Random Forest approach

In [28]:
# training RF

# Define Random Forest model
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)

#cross validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
cv_r2_scores = cross_val_score(rf_model, X_train_1, y_tr, cv=kf, scoring='r2')

# Train the model
rf_model.fit(X_train_1, y_tr)

# Predict on the training set for evaluation
y_pred_train = rf_model.predict(X_train_1)

# Calculate R^2 score on the training set
train_r2_score = r2_score(y_tr, y_pred_train)

# Print the cross-validation R^2 scores
print("Cross-Validation R^2 scores:", cv_r2_scores)

# Print the mean and standard deviation of the cross-validation R^2 scores
print("Mean CV R^2 Score:", np.mean(cv_r2_scores))
print("Standard Deviation of CV R^2 Score:", np.std(cv_r2_scores))

# Print the R^2 score on the training set
print("Training R^2 Score:", train_r2_score)


Cross-Validation R^2 scores: [0.94743308 0.9604121  0.95741922 0.96445525 0.96482397]
Mean CV R^2 Score: 0.95890872162364
Standard Deviation of CV R^2 Score: 0.006354889676654997
Training R^2 Score: 0.9949497785978473


Why random forest explanation:


In [51]:
# The Random Forest algorithm presents a promising approach for this problem.
# During the training phase, it generates numerous decision trees
# Each tree is constructed using a random subset of the dataset and measures a random subset of features in each partition.
# By constructing multiple trees with multiple subsets, it specifically learns interactions
# When a feature interacts with another feature to influence the target variable, the ensemble of trees can capture these relationships.

## 2.2 Gradient Boosting approach

In [52]:
# Gradient Boosting

from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

gb_model = GradientBoostingRegressor(n_estimators=1000, random_state=42)


num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
cv_r2_scores = cross_val_score(gb_model, X_train_1, y_tr, cv=kf, scoring='r2')


gb_model.fit(X_train_1, y_tr)
y_pred_train = gb_model.predict(X_train_1)


train_r2_score = r2_score(y_tr, y_pred_train)


print("Cross-Validation R^2 scores:", cv_r2_scores)
print("Mean CV R^2 Score:", np.mean(cv_r2_scores))
print("Standard Deviation of CV R^2 Score:", np.std(cv_r2_scores))
print("Training R^2 Score:", train_r2_score)




Cross-Validation R^2 scores: [0.9534373  0.9710533  0.96750132 0.97107206 0.9769234 ]
Mean CV R^2 Score: 0.9679974731496696
Standard Deviation of CV R^2 Score: 0.007882996660027905
Training R^2 Score: 0.9940615126747256


Why Gradient Boosting explanation

In [53]:
# Gradient Boosting is another suitable choice for this problem.
# In GB model, the process starts with simple decision tree, and newly added tree tries to explain the error left behind the previous tree.
# when feature interacts with another feature to influence the target variable, the ensemble of trees can learn these relationships.
# By minimazing loss gradient during training, it learns interactions and improve the accuracy of the model.

## 2.1.2 Random Forest with FP+previous chemical feature set

In [56]:
# training RF

import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Assuming X_train and y_train are your features and target variable respectively
# Assuming you have imported your data using data loader and preprocessed it

# Define your Random Forest model
rf_model_fp = RandomForestRegressor(n_estimators=100, random_state=42)

# Define the number of folds for cross-validation
num_folds = 2

# Initialize cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
cv_r2_scores = cross_val_score(rf_model_fp, X_train_fp, y_tr, cv=kf, scoring='r2')

# Train the model on the whole dataset
rf_model_fp.fit(X_train_fp, y_tr)

# Predict on the training set for evaluation
y_pred_train = rf_model_fp.predict(X_train_fp)

# Calculate R^2 score on the training set
train_r2_score = r2_score(y_tr, y_pred_train)

# Print the cross-validation R^2 scores
print("Cross-Validation R^2 scores:", cv_r2_scores)

# Print the mean and standard deviation of the cross-validation R^2 scores
print("Mean CV R^2 Score:", np.mean(cv_r2_scores))
print("Standard Deviation of CV R^2 Score:", np.std(cv_r2_scores))

# Print the R^2 score on the training set
print("Training R^2 Score:", train_r2_score)


Cross-Validation R^2 scores: [0.92082739 0.9575236 ]
Mean CV R^2 Score: 0.9391754962335037
Standard Deviation of CV R^2 Score: 0.018348104812371535
Training R^2 Score: 0.9944062683221588


In [57]:
# I tried using combination of fingerprint data and previous chemical features from rdkit for random forest.
# I also tried different n_estimators (100, 500, 1000), and they were 0.940 which is lower than the previous RF model.


## 2.2.2 Optimized Gradient Boosting with FP+previous chemical feature set

In [59]:
# Gradient Boosting
# Define your Gradient Boosting model
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

gb_model = GradientBoostingRegressor(n_estimators=1000, random_state=42)

# Define the number of folds for cross-validation
num_folds = 5

# Initialize cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
cv_r2_scores = cross_val_score(gb_model, X_train_fp, y_tr, cv=kf, scoring='r2')

# Train the model on the whole dataset
gb_model.fit(X_train_fp, y_tr)

# Predict on the training set for evaluation
y_pred_train = gb_model.predict(X_train_fp)

# Calculate R^2 score on the training set
train_r2_score = r2_score(y_tr, y_pred_train)

# Print the cross-validation R^2 scores
print("Cross-Validation R^2 scores:", cv_r2_scores)

# Print the mean and standard deviation of the cross-validation R^2 scores
print("Mean CV R^2 Score:", np.mean(cv_r2_scores))
print("Standard Deviation of CV R^2 Score:", np.std(cv_r2_scores))

# Print the R^2 score on the training set
print("Training R^2 Score:", train_r2_score)


#gradient booster per molecule

Cross-Validation R^2 scores: [0.96468532 0.97728473 0.97404375 0.97637981 0.97305431]
Mean CV R^2 Score: 0.9730895828551237
Standard Deviation of CV R^2 Score: 0.004471426268787465
Training R^2 Score: 0.9961560975248541


In [60]:
# I also did the same thing for gradient boosting model.
# for num_estimators, 1000 showed the highest R^2 score, which is 0.973.

## 2.4 Graph based approach 


A SMILES to graph conversion function.

In [61]:
def smiles2graph(smiles):
    '''
    Transform smiles into a list of atomic numbers and an edge array

    Args:
        smiles (str): SMILES strings

    Returns:
        z(np.array), A (np.array): list of atomic numbers, edge array
    '''

    mol = Chem.MolFromSmiles( smiles ) # no hydrogen
    z = np.array( [atom.GetAtomicNum() for atom in mol.GetAtoms()] )
    A = np.stack(Chem.GetAdjacencyMatrix(mol)).nonzero()

    return z, A


In [62]:
solvent_tr_AtomicNum_list = []
solvent_tr_Edge_list = []
solvent_tr_Natom_list = []

solute_tr_AtomicNum_list = []
solute_tr_Edge_list = []
solute_tr_Natom_list = []

solute_te_AtomicNum_list = []
solute_te_Edge_list = []
solute_te_Natom_list = []

solvent_te_AtomicNum_list = []
solvent_te_Edge_list = []
solvent_te_Natom_list = []



for smiles in train_data['Solvent']:
        z, a = smiles2graph(smiles)

        mol = Chem.MolFromSmiles( smiles )
        solvent_tr_AtomicNum_list.append(torch.LongTensor(z))
        solvent_tr_Edge_list.append(torch.LongTensor(a))
        solvent_tr_Natom_list.append(len(z))

for smiles in train_data['Solute']:
        z, a = smiles2graph(smiles)
        mol = Chem.MolFromSmiles( smiles )
        solute_tr_AtomicNum_list.append(torch.LongTensor(z))
        solute_tr_Edge_list.append(torch.LongTensor(a))
        solute_tr_Natom_list.append(len(z))

for smiles in test_data['Solvent']:
        z, a = smiles2graph(smiles)
        mol = Chem.MolFromSmiles( smiles )
        solute_te_AtomicNum_list.append(torch.LongTensor(z))
        solute_te_Edge_list.append(torch.LongTensor(a))
        solute_te_Natom_list.append(len(z))

for smiles in test_data['Solute']:
        z, a = smiles2graph(smiles)
        mol = Chem.MolFromSmiles( smiles )
        solute_te_AtomicNum_list.append(torch.LongTensor(z))
        solute_te_Edge_list.append(torch.LongTensor(a))
        solute_te_Natom_list.append(len(z))

<ipython-input-62-80beb0e00bff>:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  solvent_tr_Edge_list.append(torch.LongTensor(a))
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:41:36] WARNING: not removing hydrogen atom without neighbors
[04:4

In [63]:
class GraphDataset(torch.utils.data.Dataset):
    def __init__(self,
                 AtomicNum_list,
                 Edge_list,
                 Natom_list,
                 y_list=None):

        '''
        GraphDataset object

        Args:
            z_list (list of torch.LongTensor)
            a_list (list of torch.LongTensor)
            N_list (list of int)
            y_list (list of torch.FloatTensor), optional
                Properties to predict. Default is None.

        '''
        self.AtomicNum_list = AtomicNum_list  # atomic number
        self.Edge_list = Edge_list  # edge list
        self.Natom_list = Natom_list  # Number of atoms
        self.y_list = y_list  # properties to predict

    def __len__(self):
        return len(self.Natom_list)

    def __getitem__(self, idx):



        AtomicNum = torch.LongTensor(self.AtomicNum_list[idx])
        Edge = torch.LongTensor(self.Edge_list[idx])
        Natom = self.Natom_list[idx]

        if self.y_list is not None:
            y = self.y_list[idx]
            return AtomicNum, Edge, Natom, y
        else:
            return AtomicNum, Edge, Natom


In [73]:
# combine solvent and solute for train and test data
combined_an_tr = solvent_tr_AtomicNum_list + solute_tr_AtomicNum_list
combined_e_tr = solvent_tr_Edge_list + solute_tr_Edge_list
combined_n_tr = solvent_tr_Natom_list + solute_tr_Natom_list
y_g_tr = []
for logK in train_data['logK']:
    y_g_tr.append(torch.Tensor([logK]))
print(y_g_tr)
combined_an_te = solvent_te_AtomicNum_list + solute_te_AtomicNum_list
combined_e_te = solvent_te_Edge_list + solute_te_Edge_list
combined_n_te = solvent_te_Natom_list + solute_te_Natom_list

[tensor([4.8300]), tensor([2.7600]), tensor([3.1340]), tensor([3.4400]), tensor([2.5400]), tensor([2.3170]), tensor([2.4760]), tensor([2.4620]), tensor([1.3500]), tensor([-0.2900]), tensor([3.9470]), tensor([3.1800]), tensor([3.3500]), tensor([1.5400]), tensor([2.7010]), tensor([3.7200]), tensor([2.9880]), tensor([4.2110]), tensor([2.5800]), tensor([2.8800]), tensor([3.5560]), tensor([4.7000]), tensor([2.5570]), tensor([2.8280]), tensor([2.9400]), tensor([2.9900]), tensor([1.7960]), tensor([4.9730]), tensor([2.1280]), tensor([3.6700]), tensor([-0.3800]), tensor([3.5000]), tensor([4.7700]), tensor([-1.0700]), tensor([2.8720]), tensor([3.6400]), tensor([4.1390]), tensor([5.1000]), tensor([2.1600]), tensor([2.8680]), tensor([1.5500]), tensor([2.5290]), tensor([3.0600]), tensor([3.1740]), tensor([2.5730]), tensor([3.8600]), tensor([4.6600]), tensor([3.1520]), tensor([4.6190]), tensor([-0.8800]), tensor([2.7100]), tensor([3.6700]), tensor([3.2420]), tensor([5.2300]), tensor([1.5400]), tenso

In [74]:
X_g_tr = GraphDataset(combined_an_tr, combined_e_tr, combined_n_tr, y_g_tr)
X_g_te = GraphDataset(combined_an_te, combined_e_te, combined_n_te)

In [66]:
def collate_graphs(batch):
    '''Batch multiple graphs into one batched graph

    Args:

        batch (tuple): tuples of AtomicNum, Edge, Natom and y obtained from GraphDataset.__getitem__()

    Return
        (tuple): Batched AtomicNum, Edge, Natom, y

    '''

    AtomicNum_batch = []
    Edge_batch = []
    Natom_batch = []
    y_batch = []

    cumulative_atoms = np.cumsum([0] + [b[2] for b in batch])[:-1]

    for i in range(len(batch)):
        z, a, N, y = batch[i]
        if i >= len(X_g_tr):
            return None, None, None, None
        index_shift = cumulative_atoms[i]
        a = a + index_shift
        AtomicNum_batch.append(z)
        Edge_batch.append(a)


In [67]:

train_g_loader = DataLoader(X_g_tr,
                          batch_size=512,
                          collate_fn=collate_graphs,shuffle=True)
test_g_loader = DataLoader(X_g_te,
                          batch_size=512,
                          collate_fn=collate_graphs,shuffle=False)

In [68]:

class GNN(torch_geometric.nn.models.GCN):
    def __init__(self, n_convs=3, n_embed=64):
        # Intatiate GCN
        super(GNN, self).__init__(-1,n_embed,n_convs)
        # Declare 1 hot encoding
        self.atom_embed = nn.Embedding(100, n_embed)
        # Declare readout layers
        self.readout = nn.Sequential(nn.Linear(n_embed, n_embed), nn.ReLU(), nn.Linear(n_embed, 1))
    def forward(self, AtomicNum, Edge, Natom):
        # Parametrize embedding
        h = self.atom_embed(AtomicNum)
        # Graph convolutions
        h = super(GNN, self).forward(h,Edge)
        # Node wise output
        node_out = self.readout(h)
        # Split nodes back to graphs
        node_splits = torch.split(node_out, Natom)
        output = torch.stack([i.sum() for i in node_splits])
        return output


In [75]:
def loop(model, loader, epoch, evaluation=False):

    if evaluation:
        model.eval()
        mode = "eval"
    else:
        model.train()
        mode = 'train'
    batch_losses = []

    # Define tqdm progress bar
    tqdm_data = tqdm(loader, position=0, leave=False, desc='{} (epoch #{})'.format(mode, epoch))

    for data in tqdm_data:

        AtomicNumber, Edge, Natom, y = data
        AtomicNumber = AtomicNumber.to(device)
        Edge = Edge.to(device)
        y = y.to(device)
        pred = model(AtomicNumber, Edge, Natom)
        loss = (pred-y).pow(2).mean() # MSE loss

        if not evaluation:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        batch_losses.append(loss.item())

        postfix = ['batch loss={:.3f}'.format(loss.item()) ,
                   'avg. loss={:.3f}'.format(np.array(batch_losses).mean())]

        tqdm_data.set_postfix_str(' '.join(postfix))

    return np.array(batch_losses).mean()

In [84]:
from torch import optim

device = 'cuda:0'
model_gnn = GNN(n_convs=4, n_embed=128).to(device)

optimizer = optim.Adam(model_gnn.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=50)

for epoch in range(10):
    train_loss = loop(model_gnn, train_g_loader, epoch)
    # Optionally, update the learning rate using the scheduler
    scheduler.step(train_loss)


In [85]:
def evaluate(model, dataloader, device):

    '''
    A function to return the predicted solubility (for evaluation).

    Args:
        model (torch.nn.Module): your sequence classifier
        dataloader (torch.utils.data.Dataloader): DataLoader object for the train data
        device (str): Your device

    Returns:
        (np.array, np.array): true values, predicted values
    '''

    y_pred_list = []
    # y_list = []

    with torch.no_grad():
        model.eval()
        for batch in dataloader:

            AtomicNumber, Edge, Natom = batch
            AtomicNumber = AtomicNumber.to(device)
            Edge = Edge.to(device)
            # y = y.to(device)
            y_pred = model(AtomicNumber, Edge, Natom)

            y_pred_list+=y_pred.detach().cpu().numpy().tolist()
            # y_list+=y.detach().cpu().numpy().tolist()

    return np.array(y_pred_list)

In [86]:
y_train, y_train_pred = evaluate(model_gnn, train_g_loader, device)
train_r2 = r2_score(y_train,y_train_pred)

IndexError: list index out of range

## 2.5 Augmenting MLP model with attention mechanism

In [91]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import KFold, cross_val_score


input_tensor = torch.tensor(X_train_fp, dtype=torch.float32)
target_tensor = torch.tensor(y_tr, dtype=torch.float32)

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, input_tensor, target_tensor):
        self.input_data = input_tensor
        self.target_data = target_tensor

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return self.input_data[idx], self.target_data[idx]

# Create DataLoader for training data
train_dataset = CustomDataset(input_tensor, target_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define MLP model with attention mechanism
class MLPWithAttention(nn.Module):
    def __init__(self):
        super(MLPWithAttention, self).__init__()
        self.fc0 = nn.Linear(218, 218)
        self.fc1 = nn.Linear(218, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        attn_weights = torch.softmax(self.fc0(x), dim=1)
        x = x * attn_weights
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, criterion, and optimizer
mlp_attention_model = MLPWithAttention()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp_attention_model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for batch_input, batch_target in train_loader:
        optimizer.zero_grad()
        outputs = mlp_attention_model(batch_input)
        loss = criterion(outputs, batch_target)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
with torch.no_grad():
    output_test = mlp_attention_model(input_tensor)
    test_r2_score = r2_score(target_tensor.numpy(), output_test.numpy())

# Print the result
print("Test R^2 Score:", test_r2_score)


Test R^2 Score: 0.04590015436123629


## Conclusion

In [92]:
# In conclusion, the highest accuracy is achieved using gradient boosting with feature set of morgan fingerprint
# data and other chemical properties from rdkit.
# As we mentioned before, gradient boosting model does good job in learning interactions by
# starting with simple decision tree, and adding new tree adn tries to explain the error left behind the previous tree.
# when feature interacts with another feature to influence the target variable, the ensemble of trees can learn these relationships.
# By minimazing loss gradient during training, it learns interactions and improve the accuracy of the model.